In [17]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from sklearn.cluster import KMeans
import numpy as np

# Veri setini oluşturma
data = {'products': ["MILK,BREAD,BISCUIT", "BREAD,MILK,BISCUIT,CORNFLAKES", "BREAD,TEA,BOURNVITA",
                     "JAM,MAGGI,BREAD,MILK", "MAGGI,TEA,BISCUIT", "BREAD,TEA,BOURNVITA",
                     "MAGGI,TEA,CORNFLAKES", "MAGGI,BREAD,TEA,BISCUIT", "JAM,MAGGI,BREAD,TEA",
                     "BREAD,MILK", "COFFEE,COCK,BISCUIT,CORNFLAKES", "COFFEE,COCK,BISCUIT,CORNFLAKES",
                     "COFFEE,SUGER,BOURNVITA", "BREAD,COFFEE,COCK", "BREAD,SUGER,BISCUIT",
                     "COFFEE,SUGER,CORNFLAKES", "BREAD,SUGER,BOURNVITA", "BREAD,COFFEE,SUGER",
                     "BREAD,COFFEE,SUGER", "TEA,MILK,COFFEE,CORNFLAKES"]}
df = pd.DataFrame(data)

# Veri ön işleme
def preprocess_data(dataframe):
    transactions = [item.split(',') for item in dataframe['products']]
    te = TransactionEncoder()
    te_ary = te.fit(transactions).transform(transactions)
    df_encoded = pd.DataFrame(te_ary, columns=te.columns_)
    return df_encoded

df_encoded = preprocess_data(df)

# Apriori algoritması ile destek ve güven değerlerini bulma
frequent_itemsets = apriori(df_encoded, min_support=0.2, use_colnames=True)

# Simple KMeans algoritması için veri hazırlığı
X = df_encoded.values

# KMeans modelini oluşturup eğitme
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(X)

# Her bir örneğin ait olduğu küme numarasını alma
cluster_labels = kmeans.labels_

# Her bir örneğin ait olduğu küme numarasını DataFrame'e ekleme
df_encoded['Cluster'] = cluster_labels

# Her küme için örnek sayısını yazdırma
print("Cluster Counts:")
print(df_encoded['Cluster'].value_counts())

# Küme merkezlerini yazdırma
print("\nCluster Centers:")
print(pd.DataFrame(kmeans.cluster_centers_, columns=df_encoded.columns[:-1]))  # "Cluster" sütununu düşürdük

# Her bir özellik için küme merkez değerlerini yazdırma
print("\nCluster Center Values:")
for cluster_num, center_values in enumerate(kmeans.cluster_centers_):
    print(f"\n--- Cluster {cluster_num + 1} ---")
    for feature_name, center_value in zip(df_encoded.columns, center_values):
        print(f"{feature_name}: {center_value}")


C:\Users\DOGUKAN\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Cluster Counts:
1    10
2     7
0     3
Name: Cluster, dtype: int64

Cluster Centers:
    BISCUIT  BOURNVITA     BREAD      COCK        COFFEE  CORNFLAKES  \
0  0.666667   0.000000  0.000000  0.666667  1.000000e+00    1.000000   
1  0.400000   0.200000  0.800000  0.000000  5.551115e-17    0.200000   
2  0.142857   0.285714  0.714286  0.142857  7.142857e-01    0.142857   

            JAM  MAGGI      MILK         SUGER           TEA  
0  0.000000e+00    0.0  0.333333  5.551115e-17  3.333333e-01  
1  2.000000e-01    0.5  0.400000  0.000000e+00  6.000000e-01  
2  1.387779e-17    0.0  0.000000  8.571429e-01 -5.551115e-17  

Cluster Center Values:

--- Cluster 1 ---
BISCUIT: 0.6666666666666666
BOURNVITA: 0.0
BREAD: 0.0
COCK: 0.6666666666666666
COFFEE: 0.9999999999999999
CORNFLAKES: 0.9999999999999998
JAM: 0.0
MAGGI: 0.0
MILK: 0.3333333333333333
SUGER: 5.551115123125783e-17
TEA: 0.3333333333333333

--- Cluster 2 ---
BISCUIT: 0.4
BOURNVITA: 0.2
BREAD: 0.8
COCK: 0.0
COFFEE: 5.551115123125783e-